## 01. 프롬프트(Prompt)

PromptTemplate

In [ ]:
!pip install -qU langchain-teddynote

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = userdata.get("LANGSMITH_API_KEY")

이 코드는 API 키를 직접 `os.environ` 딕셔너리에 넣어주기 때문에, `load_dotenv()`를 사용할 필요가 전혀 없습니다.

In [ ]:
from dotenv import load_dotenv
# 이 코드는 Colab 환경에서는 불필요합니다.
load_dotenv()
# Colab에 .env 파일이 없으므로 False 반환

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LLM 객체를 정의합니다.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

## RAG 프롬프트 구조와 기본 PromptTemplate

PromptTemplate: {Question}이나 {Context} 같은 변수를 동적으로 채워 넣어 프롬프트를 생성하는 기본 도구

1. PromptTemplate.from_template()은 가장 간단하게 템플릿을 생성
2. PromptTemplate() 생성자는 input_variables를 명시하여 변수 유효성 검사를 강화

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

In [ ]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

Template(안에 우리가 받을 값을 변수로 가짐)-- from_template() 메소드--> prompt 제작 후

chain으로 llm과 연결.

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [ ]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성

추가 유효성 검사를 위해 input_variables 를 명시적으로 지정하세요.

이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킵니다.

`PromptTemplate()`으로 prompt 객체 생성

In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"], #!!
)

prompt

In [ ]:
# prompt 생성
prompt.format(country="대한민국")

## Partial Variables를 이용한 유연성 확보

### `partial_variables:` 부분 변수 채움
프롬프트 템플릿의 일부 변수에 미리 값을 할당해두는 기능

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

In [ ]:
prompt.format(country1="대한민국")

In [ ]:
#  partial_variables의 country2가 미국->캐나다 변경
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

`promt.partial()`은
- 새로운 변수 자리를 생성(X)
- 이미 템플릿에 정의되어 있는 변수에 대해 미리 값을 할당하는 역할(O)
- 원래 prompt 객체 자체의 내부 내용을 수정하지 않음. 대신, 지정된 변수에 초기값이 할당된 새로운 PromptTemplate 객체를 생성해서 반환함.

In [ ]:
prompt_partial.format(country1="대한민국")

`format()` 메소드는 해당하는 변수에 값을 할당

In [ ]:
chain = prompt_partial | llm
chain.invoke("대한민국").content

In [ ]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

### 사용 사례: **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우 (**날짜나 시간**)
`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것.

예시:
날짜를 반환하는 함수 를 사용하여 프롬프트를 부분적으로 변경

In [ ]:
#오늘 날짜를 구하는 파이썬 코드
from datetime import datetime

# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [ ]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [ ]:
# prompt 생성
prompt.format(n=3)

In [ ]:
# chain 을 생성합니다.
chain = prompt | llm

# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3).content)

In [ ]:
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

`invoke` 메소드는

 단순히 LLM의 텍스트 응답뿐만 아니라, API 호출과 관련된 다양한 메타데이터와 정보를 함께 반환.

- content: 이것이 LLM이 생성한 실제 텍스트 응답입니다. "오늘이 생일인 유명인 3명"에 대한 내용이 여기에 들어있습니다.
- additional_kwargs: 추가적인 키워드 인자나 정보가 담길 수 있는 딕셔너리입니다. 여기서는 refusal: None으로 특별한 추가 정보는 없습니다.
- response_metadata: LLM 모델 자체와 관련된 상세 정보입니다.
  - token_usage: 요청 및 응답에 사용된 토큰 수 (비용 계산 등에 사용될 수 있습니다).
  - model_name: 사용된 LLM 모델의 이름 (예: gpt-3.5-turbo-0125).
  - finish_reason: 응답 생성이 왜 중단되었는지 (예: stop은 정상적인 완료를 의미합니다).
  - 그 외 ID, 서비스 티어 등 API 호출과 관련된 정보가 포함됩니다.
- id: 이 특정 호출에 대한 고유 식별자입니다.
- usage_metadata: 전체 호출의 사용량 요약 정보입니다.

### 파일로부터 template 읽어오기

주피터 노트북을 사용할 경우 파일을 읽어오는 방법-->

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')


Drive 마운트가 완료되면, Google Drive에 업로드한 `fruit_color.yaml` 파일의 경로를 확인해야 합니다. 일반적으로 `/content/drive/MyDrive/` 아래에 파일이 위치하게 됩니다.

In [ ]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("/content/drive/MyDrive/Colab Notebooks/Langchain/fruit_color.yaml")
prompt

In [ ]:
prompt.format(fruit="사과")

In [ ]:
prompt2 = load_prompt("/content/drive/MyDrive/Colab Notebooks/Langchain/capital.yaml")
print(prompt2.format(country="대한민국"))

### ChatPromptTemplate

`ChatPromptTemplate`: 대화목록을 프롬프트로 주입하고자 할 때

메시지는 튜플(tuple) 형식으로 구성하며, (`role, message`) 로 구성하여 리스트로 생성할 수 있습니다.
  - tuple: 순서가 정해진 값들의 집합

**role**
- "system": 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
- "human" : 사용자 입력 메시지 입니다.
- "ai": AI 의 답변 메시지입니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

In [ ]:
chat_prompt.format(country="대한민국")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # (role, message) <- tuple 한개, 여기 총 4개
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

In [ ]:
llm.invoke(messages).content

이번에는 체인을 생성해 보겠습니다.

In [ ]:
chain = chat_template | llm
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content


### `MessagePlaceholder`
긴 대화 기록이나 컨텍스트를 변수 형태로 받아와 프롬프트의 원하는 위치에 통째로 삽입 가능!


메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt


conversation 대화목록을 나중에 추가하고자 할 때 MessagesPlaceholder 를 사용할 수 있습니다.

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

In [ ]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

`StrOutputParser()`

출력 파서 중 하나로, LLM 응답의 content를 문자열로 추출하는 역할을 한다.

## 02. 퓨샷 프롬프트(FewShotPromptTemplate)

Few-Shot Prompting:

LLM에게 원하는 작업의 예시를 몇 개(Few-shot) 제공하여, 모델이 예시를 보고 학습하여 비슷한 스타일이나 형식으로 응답하도록 유도하는 방법.

 Zero-Shot Prompting (예시 없이 질문만 하는 것)이나 One-Shot Prompting (예시 하나만 주는 것)보다 모델의 성능을 향상시키는 데 도움이 될 수 있습니다.

**FewShotPromptTemplate**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성: 0->모델의 응답이 더 결정적이고 일관되게 됨.
    model_name="gpt-4-turbo",  # 모델명
)

In [ ]:
# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
answer = llm.stream(question)
print("1.")
print(answer)
print("2.")
stream_response(answer)

Q1. 위의 코드들에서는 prompt를 직접 만들어서 llm과 chain으로 연결한 후에 invoke해야했는데, FewShotPrompt 관련 코드에서는 왜 prompt를 생성하지 않고 string을 바로 llm에 넣어?
> 입력 변수가 없고 단순히 고정된 질문 하나를 LLM에게 전달하는 경우에는 굳이 PromptTemplate 객체를 생성하지 않고 프롬프트 문자열을 LLM의 `invoke나 stream` 메소드에 직접 전달해도 동작합니다.

Q2. `invoke()`와 `stream()`의 차이점?
> invoke(): LLM에게 질문을 던지고 모든 답변이 완성될 때까지 기다립니다. 답변이 완전히 준비되면, 그 완성된 답변 전체를 한 번에 받아서 반환.


> stream(): 조금씩, 실시간으로 응답의 일부분(토큰)을 받습니다

Q3. 그리고 왜 `print`를 사용하지 않고 `stream_response`라는 메소드를 사용해?
> 만약 stream()의 결과를 print()로 바로 출력하려고 하면, 스트림 객체 자체나 그 내부 표현이 출력.

> ChatOpenAI() 객체에 .stream() 메소드를 사용하면, LLM의 응답이 한 번에 도착하는 것이 아니라 실시간으로 토큰 단위로 스트리밍되어 전달됩니다. 즉, 응답이 생성되는 대로 조금씩 받아볼 수 있습니다.






In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


In [ ]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))


딕셔너리 앞에 **를 붙이면, 해당 딕셔너리의 키-값 쌍들을 **함수의 키워드 인자(keyword arguments)**로 자동으로 풀어헤쳐서 전달해 줌.

**examples[0]는 이 딕셔너리를 받아서 question="..."과 answer="..."라는 두 개의 키워드 인자로 분해함.

그래서 example_prompt.format(**examples[0])는 실제로는 다음과 같이 호출되는 것과 같습니다:


```
example_prompt.format(
    question="스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
    answer="""이 질문에 추가 질문이 필요한가요: 예.
... (생략) ...
"""
)
```



이 템플릿은 예시 데이터 리스트(examples), 각 예시의 형식을 정의하는 example_prompt, 그리고 사용자 질문이 들어갈 suffix로 구성됩니다.

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

`suffix` 사용자의 실제 질문이나 프롬프트가 들어갈 자리

```
suffix="Question:\n{question}\nAnswer:",
```
이는 Few-Shot 예시들이 모두 나열된 뒤에 "Question:\n{question}\nAnswer:" 라는 형식이 추가된다는 뜻입니다. 그리고 최종적으로 prompt.format(question=question)을 통해 사용자의 질문(question 변수)이 {question} 자리에 채워져서 LLM에게 전달될 프롬프트가 완성됩니다.

따라서 suffix는 Few-Shot 예시 패턴을 보여준 후에, **모델이 실제로 응답해야 할 사용자의 입력 부분을 정의하는 템플릿 역할**을 합니다.


In [ ]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)


In [ ]:
# 위 코드들의 최종 병합본
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)


### Example Selector
예제가 많은 경우 프롬프트에 포함할 예제를 선택

-> Example Selector는 이 작업을 담당하는 클래스입니다.

- 참고: https://python.langchain.com/v0.1/docs/modules/model_io/prompts/example_selectors/

In [ ]:
!pip install -qU langchain-chroma

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    Chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)
question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})


print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')


**`Chroma`**는 **벡터 데이터베이스(Vector Database)**의 한 종류입니다.

벡터 데이터베이스는 텍스트, 이미지, 오디오 등 다양한 데이터를 벡터(숫자 목록) 형태로 저장하고, 이 **벡터들 간의 유사성을 기반으로 빠르게 검색**하는 데 특화된 데이터베이스입니다.

위 예시에서는 Chroma를 사용하여:

examples (질문-답변 예시들)을 `OpenAIEmbeddings()`를 통해 벡터로 변환 ->
변환된 벡터들을 Chroma("example_selector", ...)라는 Chroma 데이터베이스에 저장 ->
사용자의 새로운 질문({"question": question})도 벡터로 변환한 후, Chroma 데이터베이스에 저장된 예시 벡터들과 비교하여 의미적으로 가장 유사한 예시를 찾아냄.


즉, Chroma는 Few-Shot Prompting에서 사용할 "가장 비슷한 예시"를 효율적으로 검색하기 위해 사용되는 저장소 역할을 합니다.

ExampleSelector 를 사용하여 FewShotPromptTemplate 을 생성 ->

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# 체인 생성
chain = prompt | llm

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)


### FewShotChatMessagePromptTemplate

일반 FewShot은 모든 내용을 하나의 긴 문자열로 넣는 반면, Chat Message 버전은 각 예시를 (human, ai) 쌍의 대화 메시지 목록 형태로 구성합니다.

이는 모델에게 역할을 명확히 알려주기 때문에, 저희가 구현한 '회의록 작성 전문가'나 '문장 교정 전문가'와 같은 복잡한 역할극 프롬프트에 특히 효과적입니다.

In [ ]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # *예시 목록* 여기 있음~!
    examples,
    # 여기에는 의미적 *유사성을 측정*하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 *저장*하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate( #!!
    example_selector=example_selector,
    example_prompt=example_prompt,
)
# example_selector는 전달받은 사용자 입력 (question)을 사용하여, 미리 설정된 examples 목록 중에서 **가장 유사한 예시 데이터**를 하나 또는 여러 개 선택합니다.
# few_shot_prompt는 선택된 예시 데이터를 example_prompt 틀에 넣어서 메시지 목록 형태로 변환

# example_prompt 틀: [("human", "{instruction}:\n{input}"), ("ai", "{answer}")]
# 선택된 예시 데이터의 "instruction", "input", "answer" 값이 이 틀에 채워져서 HumanMessage와 AIMessage 객체로 구성된 예시 메시지 목록이 만들어집니다.
# 이렇게 만들어진 예시 메시지 목록이 최종 프롬프트 목록의 중간에 삽입됩니다.


fewshot 예제와 example selector를 사용하여 유사한 예제 1개를 선택

In [ ]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)
# Q.example_selector 내부의 example_prompt에 우리가 추출한 유사도 높은 예시가 저장되나? NO
# 아래 텍스트에 이어서 설명

이 메소드를 호출하면 example_selector는 내부적으로 사용자의 question을 벡터화하고 Chroma DB에서 가장 유사한 예시(딕셔너리 형태)를 찾아서 그 결과를 반환합니다(위 출력물). 이 시점에서는 example_prompt 객체 자체는 아무런 변화가 없습니다.

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ( #시스템 메세지
            "system",
            "You are a helpful assistant.",
        ),
        few_shot_prompt,  #!! 여기에 유사도 높은 example 있음
        ( # Human 메시지: 입력값의 "instruction"과 "input" 값이 채워짐.
         "human", "{instruction}\n{input}"),
    ]
)

# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)


### Example Selector 의 유사도 검색 문제 해결

유사도 계산시 instruction 과 input 을 사용하고 있습니다. 하지만, instruction 만 사용하여 검색시 제대로된 유사도 결과가 나오지 않습니다.

이를 해결하기 위해 커스텀 유사도 계산을 위한 클래스를 정의합니다.

아래는 잘못 검색된 결과의 예시입니다.

In [ ]:
question = {
    "instruction": "회의록을 작성해 주세요",
}

example_selector.select_examples(question)


커스텀 예제 선택기를 사용했을 때

In [ ]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "다음 문장을 교정 작성해 주세요"})


In [ ]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [ ]:
# chain 을 생성합니다.
chain = custom_prompt | llm

In [ ]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))


## 03. LangChain Hub

LangChain Hub (깃허브 아님 주의) 에서 프롬프트를 받아서 실행하는 예제!

받아오는 방법은
- '프롬프트 repo 의 아이디 값'을 가져 올 수 있고,
- 'commit id' 를 붙여서 **특정 버전**에 대한 프롬프트를 받아올 수도 있습니다.

### Hub로부터 Prompt 받아오기

In [ ]:
from langchain import hub

# 가장 최신 버전의 프롬프트를 가져옵니다.
prompt = hub.pull("rlm/rag-prompt")
# rlm라는 사용자가 게시, rag-prompt: 해당 프롬프트의 이름

# 프롬프트 내용 출력
print(prompt)

In [ ]:
# 특정 버전의 프롬프트를 가져오려면 버전 해시를 지정하세요
prompt = hub.pull("rlm/rag-prompt:50442af1") #버전 :50442af1
prompt

### Prompt Hub 에 자신의 프롬프트 등록

In [ ]:
import os
from google.colab import userdata
from langchain.prompts import ChatPromptTemplate
from langchain import hub

# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(
    "주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: {context}\n\nSUMMARY:"
)

# 프롬프트를 허브에 업로드합니다.
hub.push("no-glass-otacku/simple-summary-korean", prompt)

Hub에 푸시할 때 LangSmith가 필요한 이유:
- LangSmith는 LangChain Hub의 기반 인프라 역할을 합니다. LangChain Hub는 LangSmith 플랫폼 위에서 운영되는 서비스의 일부입니다.
- LangSmith는 LangChain 컴포넌트(프롬프트 등)를 공유하고 관리하는 기능도 포함하고 있습니다.
- 따라서 Hub에 프롬프트를 업로드(푸시)하는 것은 LangSmith 플랫폼을 통해 이루어지며, 이때 사용자를 식별하고 권한을 확인하기 위해 LangSmith API 키를 통한 인증 과정이 필요합니다.

## 04. 개인화된 프롬프트(Hub에 업로드)

In [ ]:
# 1. 라이브러리 임포트 (아직 hub는 건드리지 않습니다)
from google.colab import userdata
import os
from langsmith import Client

# 2. API 키 설정 (가장 중요)
# LANGCHAIN_API_KEY는 반드시 Colab Secrets에 등록된 이름이어야 합니다.
api_key_value = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = api_key_value
print(f"API Key Load Success: {bool(api_key_value)}")

# 3. LangSmith Client 명시적 초기화
# 이 단계를 통해 환경 변수를 강제로 읽도록 합니다.
client = Client(api_key=api_key_value)

# 4. 트레이싱 및 프로젝트 설정
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "hub-upload-practice"

# 5. 이제 Hub를 임포트 (환경 변수가 설정된 후에 임포트되어야 함)
from langchain import hub

# Owner 지정 (당신의 닉네임)
PROMPT_OWNER = "no-glass-otacku"

# 이 셀 실행 후, 다음 셀에서 hub.push()를 실행합니다.
print("LangSmith 환경 설정 완료. 이제 hub.push()를 시도하세요.")

## LangChain에서 문서를 처리하거나 특정 Task를 수행하기 위해 자주 사용되는 프롬프트 패턴들 -6가지

### **1. Stuff Documents (문서 뭉치기)**
- 가장 간단한 문서 처리 방식으로, 모든 문서의 내용을 한 프롬프트 안에 뭉쳐 넣는(Stuff) 방식입니다.
- 개념: 검색된 모든 문서 덩어리(chunk)를 그대로 하나의 LLM 입력 프롬프트의 {context} 변수에 넣어 LLM에게 전달합니다.
 - 장점: LLM이 전체 문맥을 한 번에 파악할 수 있어 가장 정확한 결과를 얻을 수 있습니다.
 - 단점: 입력 토큰 제한(Context Window)을 쉽게 초과하여 긴 문서에는 적용할 수 없습니다.





In [ ]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "summary-stuff-documents"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt

In [ ]:
hub.push(f"{PROMPT_OWNER}{prompt_title}", prompt)

### Map Prompt

**2. Map Prompt**
- 문서를 작은 덩어리로 나눈 뒤, 각 덩어리마다 동일한 프롬프트를 적용하여 처리 (주로 Map-Reduce 체인의 Map 단계에 사용)
- 장점: 토큰 제한을 피할 수 있으며, 병렬 처리가 가능
- 단점: 최종 결과를 통합하는 추가 단계(Reduce)가 필요하며, 문서 전체의 흐름(거시적 문맥)을 놓칠 수 있음.


In [ ]:
from langchain import hub
from langchain.prompts import PromptTemplate

prompt_title = "map-prompt"

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.
Please provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format.
The summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format.
Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition.
The length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.

GIVEN DOCUMENTS:
{docs}

FORMAT:
1. main theme 1
2. main theme 2
3. main theme 3
...

CAUTION:
- DO NOT list more than 5 main themes.

Helpful Answer:
"""
prompt = PromptTemplate.from_template(prompt_template)
prompt


In [ ]:
hub.push(f"{PROMPT_OWNER}{prompt_title}", prompt)

### **3. Reduce Prompt (축소 프롬프트)**
- Map 단계에서 생성된 여러 개의 중간 결과(예: 부분 요약)를 하나로 합쳐 최종 결과를 만드는 데 사용됩니다. (주로 Map-Reduce 체인의 Reduce 단계에 사용)
- 개념: Map 단계에서 나온 결과물들(여러 개의 요약, 답변 등)을 모아 하나의 프롬프트로 만든 후, LLM에게 최종 통합/정리 작업을 요청합니다.

- 장점: 여러 개의 중간 결과를 일관성 있는 최종 결과로 만듭니다.

- 단점: 통합 과정에서 또다시 토큰 제한에 부딪힐 수 있습니다.

```
"다음은 여러 부분 요약(Part Summaries)들입니다. 이를 종합하여 하나의 응집력 있고 상세한 최종 요약(Final Summary)을 만드세요. \n\nPART SUMMARIES:\n{context}\n\nFINAL SUMMARY:"
```


In [ ]:
# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert in summary writing.
You are given numbered lists of summaries.
Extract top 10 most important insights from the summaries.
Then, write a summary of the insights in KOREAN.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer:
"""

In [ ]:
# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
prompt_template = """You are a helpful expert in summary writing. You are given lists of summaries.
Please sum up previously summarized sentences according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. MOST IMPORTANT points should be organized at the top of the list.
5. DO NOT include any unnecessary information.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer: """

### **4. Metadata Tagger (메타데이터 태그 지정)**
- 제공된 텍스트에서 구조화된 정보(메타데이터)를 추출
- 개념: 문서나 텍스트를 분석하여, 미리 정의된 스키마(예: JSON 형식)에 맞춰 이름, 날짜, 키워드, 카테고리 등의 메타데이터를 추출하도록 LLM에게 요청합니다.

- 장점: 비정형 텍스트를 검색이나 필터링에 유용한 정형 데이터로 변환



In [ ]:
prompt_template = """Given the following product review, conduct a comprehensive analysis to extract key aspects mentioned by the customer, with a focus on evaluating the product's design and distinguishing between positive aspects and areas for improvement.
Identify primary features or attributes of the product that the customer appreciated or highlighted, specifically looking for mentions related to the feel of the keys, sound produced by the keys, overall user experience, charging aspect, and the design of the product, etc.
Assess the overall tone of the review (positive, neutral, or negative) based on the sentiment expressed about these attributes.
Additionally, provide a detailed evaluation of the design, outline the positive aspects that the customer enjoyed, and note any areas of improvement or disappointment mentioned.
Extract the customer's rating of the product on a scale of 1 to 5, as indicated at the beginning of the review.
Summarize your findings in a structured JSON format, including an array of keywords, evaluations for design, satisfaction points, improvement areas, the assessed tone, and the numerical rating.

INPUT:
{input}

"""

### **5. Chain of Density 요약**
- 한 번에 완성된 요약을 만드는 것이 아니라, 점진적으로 정보를 추가하며 요약의 밀도(Density)를 높이도록 LLM을 유도하는 프롬프트 체인
- 개념: 먼저 기본적인 요약을 만들게 한 후, 다음 단계에서는 이 요약에 누락된 핵심 엔티티나 정보를 추가하여 요약을 계속 수정하고 확장하도록 반복적으로 지시합니다.
- 장점: 정보가 풍부하고 세부 사항을 놓치지 않는 '밀도 높은' 요약을 생성합니다. (기존 요약보다 정보량이 많음)


```
1단계 (기본 요약): "다음 기사를 하나의 문단으로 요약하세요."
2단계 (밀도 증가): "위 요약을 수정하여 기사의 핵심 엔티티 1~3개를 추가하세요. 요약의 길이는 유지해야 합니다. \n\nOriginal Summary:\n{summary}\n\nModified Summary:"**
```



In [ ]:
prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.

A missing entity is:
- relevant to the main story,
- specific yet concise (100 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)

### **6. RAG 문서 프롬프트**
- RAG 시스템에서 검색된 문서를 LLM에게 전달할 때 사용 (주로 Stuff, Map, Reduce 체인 내부에서 사용)
- 개념: 검색 시스템이 찾아낸 원본 문서 청크를 LLM이 답변을 생성하는 데 필요한 **'컨텍스트'**로 제공하는 역할을 합니다.

- 장점: LLM이 환각(Hallucination)을 줄이고 최신 정보나 내부 데이터를 기반으로 답변하도록 강제

```
"검색된 다음 컨텍스트를 참고하여 질문에 가장 정확하고 완전한 답변을 제공하십시오. 컨텍스트에 없는 내용은 언급하지 마십시오. \n\nCONTEXT:\n{context}\n\nQUESTION:\n{question}"
```

In [ ]:
system = """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."""

human = """#Question:
{question}

#Context:
{context}

#Answer:"""

from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

In [ ]:
hub.push(f"{PROMPT_OWNER}/{prompt_title}", prompt, parent_commit_hash="latest")
